In [1]:
using BenchmarkTools
using LinearAlgebra
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

In [2]:
square = Lar.cuboid([1,1])

([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])

In [3]:
table = Lar.apply(Lar.t(-0.5,-0.5),square)

([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]])

In [4]:
structure = Lar.Struct([repeat([table,Lar.r(pi/2)],outer=2)...])

LinearAlgebraicRepresentation.Struct(Any[([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0], ([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0]], [[-0.5; -0.5], [0.5; 0.5]], "16799635533390203990", 2, "feature")

## Vecchia funzione

In [5]:
function evalStruct(self::Lar.Struct)
	dim = Lar.checkStruct(self.body)
   	CTM, stack = Matrix{Float64}(LinearAlgebra.I, dim+1, dim+1), []
   	scene = Lar.traversal(CTM, stack, self, [])
	return scene
end

evalStruct (generic function with 1 method)

In [6]:
@btime evalStruct(structure)

  8.066 μs (36 allocations: 2.88 KiB)


2-element Array{Any,1}:
 ([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]])
 ([0.49999999999999994 -0.5 0.5 -0.49999999999999994; -0.5 -0.49999999999999994 0.49999999999999994 0.5], [[1, 2, 3, 4]])

In [7]:
@benchmark evalStruct(structure)

BenchmarkTools.Trial: 
  memory estimate:  2.88 KiB
  allocs estimate:  36
  --------------
  minimum time:     8.000 μs (0.00% GC)
  median time:      9.833 μs (0.00% GC)
  mean time:        10.655 μs (1.42% GC)
  maximum time:     1.539 ms (98.32% GC)
  --------------
  samples:          10000
  evals/sample:     3

In [8]:
@code_warntype evalStruct(structure)

Variables
  #self#::Core.Compiler.Const(evalStruct, false)
  self::LinearAlgebraicRepresentation.Struct
  dim::Any
  CTM::Array{Float64,2}
  stack::Array{Any,1}
  scene::Any

Body::Any
1 ─ %1  = Base.getproperty(Main.Lar, :checkStruct)::Any
│   %2  = Base.getproperty(self, :body)::Array
│         (dim = (%1)(%2))
│   %4  = Core.apply_type(Main.Matrix, Main.Float64)::Core.Compiler.Const(Array{Float64,2}, false)
│   %5  = LinearAlgebra.I::Core.Compiler.Const(UniformScaling{Bool}(true), false)
│   %6  = (dim + 1)::Any
│   %7  = (dim + 1)::Any
│   %8  = (%4)(%5, %6, %7)::Array{Float64,2}
│   %9  = Base.vect()::Array{Any,1}
│         (CTM = %8)
│         (stack = %9)
│   %12 = Base.getproperty(Main.Lar, :traversal)::Any
│   %13 = CTM::Array{Float64,2}
│   %14 = stack::Array{Any,1}
│   %15 = Base.vect()::Array{Any,1}
│         (scene = (%12)(%13, %14, self, %15))
└──       return scene


## Modifiche

In [40]:
function evalStruct(self::Lar.Struct)
	dim::Int = Lar.checkStruct(self.body)
	CTM = Matrix{Float64}(LinearAlgebra.I, dim+1, dim+1)
	return Lar.traversal(CTM, [], self, [])
end

evalStruct (generic function with 2 methods)

In [41]:
@btime evalStruct(structure)

  7.725 μs (36 allocations: 2.88 KiB)


2-element Array{Any,1}:
 ([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]])
 ([0.49999999999999994 -0.5 0.5 -0.49999999999999994; -0.5 -0.49999999999999994 0.49999999999999994 0.5], [[1, 2, 3, 4]])

In [42]:
@code_warntype evalStruct(structure)

Variables
  #self#::Core.Compiler.Const(evalStruct, false)
  self::LinearAlgebraicRepresentation.Struct
  dim::Int64
  CTM::Array{Float64,2}

Body::Any
1 ─ %1  = Base.getproperty(Main.Lar, :checkStruct)::Any
│   %2  = Base.getproperty(self, :body)::Array
│   %3  = (%1)(%2)::Any
│   %4  = Base.convert(Main.Int, %3)::Any
│         (dim = Core.typeassert(%4, Main.Int))
│   %6  = Core.apply_type(Main.Matrix, Main.Float64)::Core.Compiler.Const(Array{Float64,2}, false)
│   %7  = LinearAlgebra.I::Core.Compiler.Const(UniformScaling{Bool}(true), false)
│   %8  = (dim + 1)::Int64
│   %9  = (dim + 1)::Int64
│         (CTM = (%6)(%7, %8, %9))
│   %11 = Base.getproperty(Main.Lar, :traversal)::Any
│   %12 = CTM::Array{Float64,2}
│   %13 = Base.vect()::Array{Any,1}
│   %14 = Base.vect()::Array{Any,1}
│   %15 = (%11)(%12, %13, self, %14)::Any
└──       return %15
